In [126]:
import os
import pandas as pd
import numpy as np

In [2]:
derivs_dir = os.path.join('..','derivatives')
scales_dir = os.path.join(derivs_dir,'qualtrics','2.subscaled')
output_dir = os.path.join(derivs_dir,'subject-level')

In [8]:
try:os.mkdir(output_dir)
except WindowsError as e: print(e)

[Error 183] Cannot create a file when that file already exists: '..\\derivatives\\subject-level'


Import data

In [10]:
fname=os.path.join(derivs_dir,'sub-all_task-all_VALUES.xlsx')
behav_data = pd.read_excel(fname)

In [14]:
fname=os.path.join(scales_dir,'all_subscales.csv')
scale_data = pd.read_csv(fname)

In [15]:
scale_data.head()

,ssid,ZIP,Relative Sleep,Relative Stress,Financial Difficulty,PAS,NAS,BAS Drive,BAS Fun Seeking,BAS Reward Responsiveness,BIS,Intuitive DMS,Rational DMS,Dependent DMS,Spontaneous DMS,Avoidant DMS,Financial Literacy
0,201.0,00233,-1.0,1.0,1.0,32.0,17.0,12.0,8.0,20.0,20.0,18.0,22.0,21.0,11.0,16.0,1.0
1,202.0,34109,0.0,-1.0,0.0,39.0,10.0,13.0,15.0,20.0,16.0,24.0,25.0,8.0,15.0,9.0,1.0
2,203.0,33071,-1.0,0.0,0.0,33.0,13.0,13.0,13.0,20.0,25.0,18.0,20.0,25.0,15.0,23.0,1.0
3,146.0,32828,0.0,0.0,0.0,36.0,15.0,12.0,11.0,17.0,21.0,16.0,22.0,17.0,7.0,13.0,0.0
4,205.0,33027,-1.0,-1.0,1.0,26.0,25.0,14.0,12.0,19.0,24.0,23.0,23.0,22.0,14.0,14.0,2.0


Normalize PANAS and BISBAS subscales

3sd trial exclusions

In [205]:
df=behav_data[['subjnum','block','trial','domain','estimation','trueprob','val-estdiff-valid']]

In [206]:
subj_3sd = df.groupby('subjnum').std()['val-estdiff-valid'] * 3
subj_3sd.head()

subjnum
100    0.548469
101    0.262039
102    0.358415
103    0.620042
104    0.809025
Name: val-estdiff-valid, dtype: float64

In [207]:
subj_means = df.groupby('subjnum').mean()['val-estdiff-valid']
subj_means.head()

subjnum
100    0.046724
101   -0.014427
102    0.014467
103    0.036173
104    0.070082
Name: val-estdiff-valid, dtype: float64

In [208]:
def exclude_3sd(row):
    ssid,val = row['subjnum'],row['val-estdiff-valid']
    mean,bound = subj_means[ssid],subj_3sd[ssid]
    diff = abs(val - mean)
    if diff < bound:
        return(val)
    else: return(np.nan)
    

In [211]:
df['val-error-3sd'] = df.apply(exclude_3sd,axis=1)
df.head()

C:\Users\ia406477\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,subjnum,block,trial,domain,estimation,trueprob,val-estdiff-valid,val-error-3sd
0,100,1,1,LOSS,50,0.300000,2.000001e-01,2.000001e-01
1,100,1,2,LOSS,30,0.155172,1.448277e-01,1.448277e-01
2,100,1,3,LOSS,30,0.300000,1.038193e-07,1.038193e-07
3,100,1,4,LOSS,40,0.500000,-1.000000e-01,-1.000000e-01
4,100,1,5,LOSS,60,0.700000,-1.000001e-01,-1.000001e-01


subject-level means

In [238]:
domain_means = df.groupby(['subjnum','domain']).mean().reset_index()
domain_means.head()

,subjnum,domain,block,trial,trueprob,val-estdiff-valid,val-error-3sd
0,100,GAIN,6.666667,37.5,0.333548,0.102563,0.102563
1,100,LOSS,6.333333,35.5,0.473004,-0.009115,-0.009115
2,101,GAIN,7.500000,42.5,0.552187,0.003091,0.003091
3,101,LOSS,5.500000,30.5,0.500000,-0.031944,-0.031944
4,102,GAIN,8.000000,45.5,0.420043,0.034123,0.034123


In [239]:
gain_ave_val_error = domain_means[domain_means['domain'] == 'GAIN']['val-error-3sd']

In [240]:
loss_ave_val_error = domain_means[domain_means['domain'] == 'LOSS']['val-error-3sd']

In [241]:
output = pd.DataFrame({
    'ssid':list(subj_means.index),
    'ValError':list(subj_means),
    'gainValError':list(gain_ave_val_error),
    'lossValError':list(loss_ave_val_error)
})
output['valWedge'] = abs(output['gainValError'] - output['lossValError'])

In [243]:
fname=os.path.join(output_dir,'subject-level.csv')
output[[
    'ssid','ValError','gainValError','lossValError','valWedge'
]].to_csv(fname,index=False)